# Modeling : lightgbm

In [1]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

In [2]:
# PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }
# prices = pd.read_csv("../datasets/sell_prices.csv", dtype = PRICE_DTYPES)
# for col, col_dtype in PRICE_DTYPES.items():
#     if col_dtype == "category":
#         prices[col] = prices[col].cat.codes.astype("int16")  # 类似于labelencoder，将store_id和item_id数字化表示
#         prices[col] -= prices[col].min()  
# # prices.head(1)

In [3]:
# CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
#          "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
#         "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
# cal = pd.read_csv("../datasets/calendar.csv", dtype = CAL_DTYPES)
# cal["date"] = pd.to_datetime(cal["date"])  #  转化为datetime类型
# for col, col_dtype in CAL_DTYPES.items():
#     if col_dtype == "category":
#         cal[col] = cal[col].cat.codes.astype("int16")  # 将event_name_1/event_type_1/event_name_2/event_type_2weekday数字化表示
#         cal[col] -= cal[col].min()  # NaN会转换为-1，该代表将NaN归为0，其他从1开始
# # cal.head(1)

In [4]:
IS_LOCAL_DEMO = True
# IS_LOCAL_DEMO = False

In [5]:
max_lags = 57
last_day = 1913  
fday = datetime(2016,4, 25) 

In [6]:
data_file = ("train_data.csv.gz" if(IS_LOCAL_DEMO) else "train_data.csv")
dt = pd.read_csv(data_file)

print("loading:",data_file)

loading: train_data.csv.gz


In [7]:
dt["date"] = pd.to_datetime(dt["date"])  


In [8]:
df = dt.copy()

In [9]:
# %time
# first_day = 1850
# numcols = [f"d_{day}" for day in range(first_day, last_day + 1)]  # 选取历史数据，从first_day到tr_last（包含这一天）d_1200,d_1201
# catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
# dtype = {numcol: "float32" for numcol in numcols}   # d_1200为float32

# dtype.update({col: "category" for col in catcols if col != "id"})  # 除id外，都为category类型
# dt = pd.read_csv("../datasets/sales_train_validation.csv", usecols = catcols + numcols, dtype = dtype,nrows=nrows_demo)  # 取catcols和numcols的列(只取了需要的历史天数销量数据)
# dt

In [10]:
# for col in catcols:
#         if col != "id":
#             dt[col] = dt[col].cat.codes.astype("int16")  # 数字化
#             dt[col] -= dt[col].min()
    
    
# for day in range(last_day + 1, last_day + 28 +1):  # 遍历预测的每一天
#     dt[f"d_{day}"] = np.nan  # 先填充为NaN

# dt = pd.melt(dt,
#               id_vars = catcols,
#               value_vars = [col for col in dt.columns if col.startswith("d_")],
#               var_name = "d",
#               value_name = "sales")



In [11]:
# dt = dt.merge(cal, on= "d", copy = False)
# dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

In [12]:
# dt["d"]=dt["d"].apply(lambda x: int(x.split("_")[1])) # d_1 to 1


In [13]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):  # [(7, lag_7), (28, lag_28)]
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)  

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean()) 

    date_features = {
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",

    }
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [14]:
create_fea(df)

In [15]:
df.dropna(inplace = True)
df.shape

(90000, 31)

In [16]:
df.columns

Index(['id', 'item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_7', 'lag_28',
       'rmean_7_7', 'rmean_28_7', 'rmean_7_28', 'rmean_28_28', 'week',
       'quarter', 'mday'],
      dtype='object')

In [17]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday", 'event_name_1', 'event_name_2']
train_cols = df.columns[~df.columns.isin(useless_cols)]
print(train_cols)
X_train = df[train_cols]
y_train = df["sales"]

Index(['item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'wday', 'month',
       'year', 'event_type_1', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',
       'sell_price', 'lag_7', 'lag_28', 'rmean_7_7', 'rmean_28_7',
       'rmean_7_28', 'rmean_28_28', 'week', 'quarter', 'mday'],
      dtype='object')


In [26]:
np.random.seed(777)

# size = (1_000 if(IS_LOCAL_DEMO) else 1_000_000)

fake_valid_inds = np.random.choice(X_train.index.values, 1_000, replace = False)  
# print(fake_valid_inds)
print(len(fake_valid_inds))

train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds) 
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)

100


In [19]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 2000,
    'num_leaves': 128,
    "min_data_in_leaf": 50,
}

In [20]:
m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=100)

/Users/Tommy/opt/anaconda3/envs/python36/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/Tommy/opt/anaconda3/envs/python36/lib/python3.6/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	valid_0's rmse: 2.3788
[200]	valid_0's rmse: 2.38309
[300]	valid_0's rmse: 2.40137
[400]	valid_0's rmse: 2.41032
[500]	valid_0's rmse: 2.42431
[600]	valid_0's rmse: 2.4241
[700]	valid_0's rmse: 2.42064
[800]	valid_0's rmse: 2.42683
[900]	valid_0's rmse: 2.43035
[1000]	valid_0's rmse: 2.43143
[1100]	valid_0's rmse: 2.44216
[1200]	valid_0's rmse: 2.44418
[1300]	valid_0's rmse: 2.44771
[1400]	valid_0's rmse: 2.45553
[1500]	valid_0's rmse: 2.46243
[1600]	valid_0's rmse: 2.46571
[1700]	valid_0's rmse: 2.46487
[1800]	valid_0's rmse: 2.47072
[1900]	valid_0's rmse: 2.47643
[2000]	valid_0's rmse: 2.47853


In [21]:
if(IS_LOCAL_DEMO):
    alphas = [1] # for local demo
else:
    alphas = [1.035, 1.03, 1.025] #[1.015,1.25,1.03] #[1.0,1.15,1.01] #[1.0,1.15,1.01] # for kaggle

weights = [1 / len(alphas)] * len(alphas) 
print(weights)

[1.0]


In [22]:
sub = 0.
cols = [f"F{i}" for i in range(1,29)]

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):  # [(1.035, 0.333), (1.03, 0.333), (1.025, 0.333)]
    te = dt

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)  
        print(icount, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()  
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]  
        te.loc[te.date == day, "sales"] = alpha * m_lgb.predict(tst)

    te_sub = te.loc[te.date >= fday, ["id", "sales"]]
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]  # [F1,F2,F3,...F28]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()  
    te_sub.fillna(0., inplace = True) 
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)



0 2016-04-25 00:00:00
0 2016-04-26 00:00:00
0 2016-04-27 00:00:00
0 2016-04-28 00:00:00
0 2016-04-29 00:00:00
0 2016-04-30 00:00:00
0 2016-05-01 00:00:00
0 2016-05-02 00:00:00
0 2016-05-03 00:00:00
0 2016-05-04 00:00:00
0 2016-05-05 00:00:00
0 2016-05-06 00:00:00
0 2016-05-07 00:00:00
0 2016-05-08 00:00:00
0 2016-05-09 00:00:00
0 2016-05-10 00:00:00
0 2016-05-11 00:00:00
0 2016-05-12 00:00:00
0 2016-05-13 00:00:00
0 2016-05-14 00:00:00
0 2016-05-15 00:00:00
0 2016-05-16 00:00:00
0 2016-05-17 00:00:00
0 2016-05-18 00:00:00
0 2016-05-19 00:00:00
0 2016-05-20 00:00:00
0 2016-05-21 00:00:00
0 2016-05-22 00:00:00
0 1 1.0


In [23]:
sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)

In [24]:
print(sub.shape)
print(sub.columns)
sub.head(3)

(20000, 29)
Index(['id', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'],
      dtype='object', name='F')


F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.882484,0.637337,0.562766,0.412774,1.037201,0.605822,0.923160,1.354341,0.865347,...,0.994954,1.378011,1.984383,2.968749,2.146219,1.748646,1.117743,1.193438,1.212587,1.692615
1,FOODS_1_001_CA_2_validation,1.846428,1.633850,1.293822,1.306879,1.056510,1.562057,2.549772,1.568928,1.541383,...,1.024316,1.481218,2.501897,0.757024,1.418542,0.967959,1.409757,0.633885,0.854505,2.701530
2,FOODS_1_001_CA_3_validation,0.363500,0.281384,0.311208,0.280363,0.282572,1.368990,2.009802,1.520692,1.505360,...,0.598873,0.958998,1.672139,0.671926,0.572143,0.587727,0.542686,0.418978,0.451443,1.836137


In [25]:
if(IS_LOCAL_DEMO):
    sub.to_csv("submission_lgb.csv.gz",compression='gzip',index=False)# for local demo
else:
    sub.to_csv("submission_lgb.csv",index=False) # for kaggle  

* ## LGB kaggle result ： 0.22838